# Introduction to Iris

* from https://scitools.org.uk/iris/docs/latest/ : "Iris implements a data model based on the CF conventions giving you a powerful, format-agnostic interface for working with your data. It excels when working with multi-dimensional Earth Science data, where tabular representations become unwieldy and inefficient. "
* **Iris** is useful for using with Netcdf files, if they are written with CF conventions in mind. 
* The MetOffice uses Iris with their fieldsfiles (.pp)


First let's do some importing

In [1]:
import iris # import the Iris module

In [2]:
import matplotlib.pyplot as plt # matplotlib is going to be useful later

To start we will explore the principal data structure used in Iris: The Cube. We will explore how to call, write, and replace attributes, data, and coordinates contained within a Cube.

## Loading cubes

There are different ways to load cubes: load(), load_cube() are the two main functions I use, and each work a little differently:

In [8]:
path_to_data = '../data/4p4km_air_potential_temperature_1.nc' # just the path to where we keep our data file plus the file name
cubes = iris.load(path_to_data)

In [9]:
print(cubes)

0: air_potential_temperature / (K)     (time: 12; model_level_number: 71; grid_latitude: 340; grid_longitude: 340)
1: surface_altitude / (m)              (grid_latitude: 340; grid_longitude: 340)


In [10]:
print(cubes[0])

air_potential_temperature                    (time: 12; model_level_number: 71; grid_latitude: 340; grid_longitude: 340)
     Dimension coordinates:
          time                                    x                       -                  -                    -
          model_level_number                      -                       x                  -                    -
          grid_latitude                           -                       -                  x                    -
          grid_longitude                          -                       -                  -                    x
     Auxiliary coordinates:
          forecast_period                         x                       -                  -                    -
          atmosphere_hybrid_height_coordinate     -                       x                  -                    -
          sigma                                   -                       x                  -                    -
          s

In [14]:
cube = iris.load_cube(path_to_data, 'air_potential_temperature')
cube

Air Potential Temperature (K),time,model_level_number,grid_latitude,grid_longitude
Shape,12,71,340,340
Dimension coordinates,,,,
time,x,-,-,-
model_level_number,-,x,-,-
grid_latitude,-,-,x,-
grid_longitude,-,-,-,x
Auxiliary coordinates,,,,
forecast_period,x,-,-,-
atmosphere_hybrid_height_coordinate,-,x,-,-
sigma,-,x,-,-
